## You're ready to put into practice everything that you've learned so far. Here are the next steps for your capstone:







```r
library(xlsx)
#fileUrl <- "http://www.aussportsbetting.com/historical_data/nfl.xlsx" # money line, open/close lines historic info
#download.file(fileUrl)
#nfl_asb <- read.csv("nfl_2014_2017_asb.csv") 

# pro football reference game info
library(XML)
library(RCurl)
library(rvest)
#filename <- NA
#for(year in 1966:2017){
#        filename[year] <- paste("https://www.pro-football-reference.com/years/",year,"/games.htm#games::none",sep="")
#} # read seasons 1966 to 2017
#url_pfr_games <- getURL(filename[1966:2017])  #getURL("https://www.pro-football-reference.com/years/2014/games.htm#games::none")
#pfr_games_raw <- readHTMLTable(url_pfr_games, trim=T, as.data.frame=T, header=T)
#pfr_games <-bind_rows(pfr_games_raw)
#my_df <- as.data.frame(read_html(url_pfr_games) %>% html_table(fill=TRUE))
```

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 1) Go out and find a dataset of interest. It could be from one of the recommended resources or some other aggregation. Or it could be something that you scraped yourself. Just make sure that it has lots of variables, including an outcome of interest to you.

In [2]:
all_dfs = []

# read seasons 1966 to 2017
for year in range(1966,2018):
    print(year)
    url = "https://www.pro-football-reference.com/years/" + str(year) + "/games.htm#games::none"
    df = pd.read_html(url)
    df = df[0]
    df['Year'] = year

    all_dfs.append(df)
    
scores1_df = pd.concat(all_dfs)
scores1_df.shape

1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017


(12743, 15)

In [3]:
scores1_df.shape

(12743, 15)

In [4]:
odds_df = pd.read_excel("http://www.aussportsbetting.com/historical_data/nfl.xlsx")
scores2_df = pd.read_csv('data/spreadspoke_scores.csv', encoding = "ISO-8859-1", engine='python')
stadiums_df = pd.read_csv('data/nfl_stadiums.csv', encoding = "ISO-8859-1", engine='python')
teams_df = pd.read_csv('data/nfl_teams.csv', encoding = "ISO-8859-1", engine='python')

## 2) Explore the data. Get to know the data. Spend a lot of time going over its quirks. You should understand how it was gathered, what's in it, and what the variables look like.

In [5]:
scores1_df.head()

,Week,Day,Date,Time,Winner/tie,Unnamed: 5,Loser/tie,Unnamed: 7,PtsW,PtsL,YdsW,TOW,YdsL,TOL,Year
0,1,Sat,September 10,NaN,Green Bay Packers,NaN,Baltimore Colts,boxscore,24,3,292,1,213,3,1966
1,1,Sun,September 11,NaN,Los Angeles Rams,@,Atlanta Falcons,boxscore,19,14,421,2,237,2,1966
2,1,Sun,September 11,NaN,Detroit Lions,NaN,Chicago Bears,boxscore,14,3,208,3,256,3,1966
3,1,Sun,September 11,NaN,San Francisco 49ers,NaN,Minnesota Vikings,boxscore,20,20,307,1,298,1,1966
4,1,Sun,September 11,NaN,Pittsburgh Steelers,NaN,New York Giants,boxscore,34,34,404,3,279,5,1966


In [6]:
scores1_df.shape

(12743, 15)

In [7]:
scores1_df.isna().mean()

Week          0.004081
Day           0.004081
Date          0.000000
Time          0.039787
Winner/tie    0.004081
Unnamed: 5    0.603155
Loser/tie     0.004081
Unnamed: 7    0.065134
PtsW          0.004081
PtsL          0.004081
YdsW          0.004081
TOW           0.004081
YdsL          0.004081
TOL           0.004081
Year          0.000000
dtype: float64

In [8]:
scores1_df = scores1_df.drop(columns=['Unnamed: 5', 'Unnamed: 7', 'Time'])

In [9]:
scores1_df = scores1_df.dropna()

In [10]:
scores1_df['Date'].value_counts()

Date           778
November 23    124
December 24    119
November 22    114
December 14    114
              ... 
February 7       2
February 6       2
February 4       2
January 29       1
February 2       1
Name: Date, Length: 162, dtype: int64

In [11]:
mos = {'January': 1
       ,'February': 2
       ,'March': 3
       ,'April': 4
       ,'May': 5
       ,'June': 6
       ,'July': 7
       ,'August': 8
       ,'September': 9
       ,'October': 10
       ,'November': 11
       ,'December': 12}

In [12]:
match = 'January'
for m in mos:
    if m != 'January':
        match = match + '|' + m
scores1_df = scores1_df[scores1_df['Date'].str.contains(match)]

In [13]:
scores1_df['Month'] = scores1_df['Date'].str.split(' ').str[0]
scores1_df['DayofMonth'] = scores1_df['Date'].str.split(' ').str[1].astype(int)

scores1_df['month_num'] = scores1_df['Month'].map(mos)

In [14]:
scores1_df

,Week,Day,Date,Winner/tie,Loser/tie,PtsW,PtsL,YdsW,TOW,YdsL,TOL,Year,Month,DayofMonth,month_num
0,1,Sat,September 10,Green Bay Packers,Baltimore Colts,24,3,292,1,213,3,1966,September,10,9
1,1,Sun,September 11,Los Angeles Rams,Atlanta Falcons,19,14,421,2,237,2,1966,September,11,9
2,1,Sun,September 11,Detroit Lions,Chicago Bears,14,3,208,3,256,3,1966,September,11,9
3,1,Sun,September 11,San Francisco 49ers,Minnesota Vikings,20,20,307,1,298,1,1966,September,11,9
4,1,Sun,September 11,Pittsburgh Steelers,New York Giants,34,34,404,3,279,5,1966,September,11,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,Division,Sun,January 14,Jacksonville Jaguars,Pittsburgh Steelers,45,42,378,0,545,2,2017,January,14,1
280,Division,Sun,January 14,Minnesota Vikings,New Orleans Saints,29,24,403,1,358,2,2017,January,14,1
281,ConfChamp,Sun,January 21,New England Patriots,Jacksonville Jaguars,24,20,344,1,374,0,2017,January,21,1
282,ConfChamp,Sun,January 21,Philadelphia Eagles,Minnesota Vikings,38,7,456,0,333,3,2017,January,21,1


## --------------------------------

In [15]:
odds_df.head()

,Date,Home Team,Away Team,Home Score,Away Score,Overtime?,Playoff Game?,Neutral Venue?,Home Odds Open,Home Odds Min,...,Total Score Close,Total Score Over Open,Total Score Over Min,Total Score Over Max,Total Score Over Close,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close,Notes
0,2020-09-14,Denver Broncos,Tennessee Titans,14,16,NaN,NaN,NaN,1.76,1.76,...,41.0,1.9,1.86,1.9,1.9,1.9,1.9,1.9,1.9,NaN
1,2020-09-14,New York Giants,Pittsburgh Steelers,16,26,NaN,NaN,NaN,2.60,2.60,...,45.0,1.9,1.90,1.9,1.9,1.9,1.9,1.9,1.9,NaN
2,2020-09-13,Los Angeles Rams,Dallas Cowboys,20,17,NaN,NaN,NaN,2.15,2.15,...,51.5,1.9,1.90,1.9,1.9,1.9,1.9,1.9,1.9,NaN
3,2020-09-13,New Orleans Saints,Tampa Bay Buccaneers,34,23,NaN,NaN,NaN,1.47,1.47,...,47.5,1.9,1.90,1.9,1.9,1.9,1.9,1.9,1.9,NaN
4,2020-09-13,San Francisco 49ers,Arizona Cardinals,20,24,NaN,NaN,NaN,1.28,1.28,...,48.0,1.9,1.90,1.9,1.9,1.9,1.9,1.9,1.9,NaN


In [16]:
odds_df.shape

(3754, 45)

In [17]:
odds_df.describe()

,Home Score,Away Score,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,Away Odds Max,Away Odds Close,...,Total Score Max,Total Score Close,Total Score Over Open,Total Score Over Min,Total Score Over Max,Total Score Over Close,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close
count,3754.000000,3754.000000,3754.000000,1618.00000,1618.000000,1618.000000,3754.000000,1618.000000,1618.000000,1618.000000,...,1618.000000,1618.000000,1618.000000,1618.000000,1618.000000,1618.000000,1618.000000,1618.000000,1618.000000,1618.000000
mean,23.479489,21.238679,1.950855,1.83207,2.064315,1.954741,2.892580,2.573298,2.982897,2.780451,...,46.400494,45.441595,1.937429,1.903295,1.967795,1.933845,1.932285,1.924482,1.942302,1.936200
std,10.380618,10.035696,0.999993,0.79537,1.035032,0.912600,1.795221,1.342111,1.692837,1.560962,...,4.115823,4.115203,0.034949,0.033771,0.057890,0.041980,0.034153,0.033072,0.041815,0.043195
min,0.000000,0.000000,1.011111,1.02000,1.050000,1.020000,1.055556,1.030000,1.060000,1.050000,...,37.000000,35.000000,1.806000,1.740000,1.860000,1.740000,1.775000,1.830000,1.787000,1.813000
25%,16.000000,14.000000,1.340000,1.33300,1.416000,1.377000,1.695250,1.641000,1.813000,1.699000,...,43.500000,42.500000,1.900000,1.884000,1.900000,1.900000,1.900000,1.900000,1.900000,1.900000
50%,23.000000,21.000000,1.606061,1.57050,1.714000,1.640000,2.450000,2.250000,2.555000,2.400000,...,46.000000,45.000000,1.952000,1.900000,1.961000,1.925000,1.952000,1.909000,1.952000,1.925000
75%,30.000000,27.000000,2.250000,2.09000,2.397500,2.260000,3.480000,3.100000,3.560000,3.310000,...,49.000000,48.000000,1.952000,1.917000,2.010000,1.961000,1.952000,1.952000,1.970000,1.961000
max,62.000000,59.000000,12.000000,10.00000,15.500000,11.000000,31.000000,12.250000,16.000000,16.000000,...,63.500000,62.500000,2.170000,2.020000,2.170000,2.110000,2.120000,2.120000,2.090000,2.220000


In [18]:
odds_df.isna().mean()

Date                       0.000000
Home Team                  0.000000
Away Team                  0.000000
Home Score                 0.000000
Away Score                 0.000000
Overtime?                  0.941929
Playoff Game?              0.958977
Neutral Venue?             0.988013
Home Odds Open             0.000000
Home Odds Min              0.568993
Home Odds Max              0.568993
Home Odds Close            0.568993
Away Odds Open             0.000000
Away Odds Min              0.568993
Away Odds Max              0.568993
Away Odds Close            0.568993
Home Line Open             0.000000
Home Line Min              0.568993
Home Line Max              0.568993
Home Line Close            0.568993
Away Line Open             0.568993
Away Line Min              0.568993
Away Line Max              0.568993
Away Line Close            0.568993
Home Line Odds Open        0.568993
Home Line Odds Min         0.568993
Home Line Odds Max         0.568993
Home Line Odds Close       0

In [19]:
odds_df['Overtime?'].value_counts()

Y    218
Name: Overtime?, dtype: int64

In [20]:
odds_df['Playoff Game?'].value_counts()

Y    154
Name: Playoff Game?, dtype: int64

In [21]:
odds_df['Neutral Venue?'].value_counts()

Y    45
Name: Neutral Venue?, dtype: int64

In [22]:
odds_df['Overtime?'] = odds_df['Overtime?'].fillna('N')
odds_df['Playoff Game?'] = odds_df['Playoff Game?'].fillna('N')
odds_df['Neutral Venue?'] = odds_df['Neutral Venue?'].fillna('N')

In [23]:
odds_cols = ['Date'                       
            ,'Home Team'                 
            ,'Away Team'                 
            ,'Home Score'                
            ,'Away Score'               
            ,'Overtime?'               
            ,'Playoff Game?'            
            ,'Neutral Venue?'           
            ,'Home Odds Open'             
            ,'Away Odds Open'           
            ,'Total Score Open']         
odds_df = odds_df[odds_cols]

In [24]:
odds_df['date'] = pd.to_datetime(odds_df['Date'])
odds_df['year'] = odds_df['date'].dt.year
odds_df['month'] = odds_df['date'].dt.month
odds_df['DayofMonth'] = odds_df['date'].dt.day

In [25]:
odds_df

,Date,Home Team,Away Team,Home Score,Away Score,Overtime?,Playoff Game?,Neutral Venue?,Home Odds Open,Away Odds Open,Total Score Open,date,year,month,DayofMonth
0,2020-09-14,Denver Broncos,Tennessee Titans,14,16,N,N,N,1.760000,2.100000,42.0,2020-09-14,2020,9,14
1,2020-09-14,New York Giants,Pittsburgh Steelers,16,26,N,N,N,2.600000,1.550000,48.0,2020-09-14,2020,9,14
2,2020-09-13,Los Angeles Rams,Dallas Cowboys,20,17,N,N,N,2.150000,1.740000,50.0,2020-09-13,2020,9,13
3,2020-09-13,New Orleans Saints,Tampa Bay Buccaneers,34,23,N,N,N,1.470000,2.750000,49.5,2020-09-13,2020,9,13
4,2020-09-13,San Francisco 49ers,Arizona Cardinals,20,24,N,N,N,1.280000,3.750000,46.0,2020-09-13,2020,9,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3749,2006-09-10,Detroit Lions,Seattle Seahawks,6,9,N,N,N,3.500000,1.333333,44.0,2006-09-10,2006,9,10
3750,2006-09-10,Cleveland Browns,New Orleans Saints,14,19,N,N,N,1.500000,2.750000,36.5,2006-09-10,2006,9,10
3751,2006-09-10,Carolina Panthers,Atlanta Falcons,6,20,N,N,N,1.434783,2.900000,38.5,2006-09-10,2006,9,10
3752,2006-09-10,Arizona Cardinals,San Francisco 49ers,34,27,N,N,N,1.181818,5.000000,43.5,2006-09-10,2006,9,10


## --------------------------------

In [26]:
scores2_df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45,7,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,14,19,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62,NaN


In [27]:
scores2_df.isna().mean()

schedule_date          0.000000
schedule_season        0.000000
schedule_week          0.000000
schedule_playoff       0.000000
team_home              0.000000
score_home             0.000000
score_away             0.000000
team_away              0.000000
team_favorite_id       0.195536
spread_favorite        0.195536
over_under_line        0.196324
stadium                0.000000
stadium_neutral        0.000000
weather_temperature    0.058527
weather_wind_mph       0.058527
weather_humidity       0.338381
weather_detail         0.791844
dtype: float64

In [28]:
scores2_df = scores2_df.drop(columns = ['weather_humidity', 'weather_detail'])

In [29]:
scores2_df = scores2_df.dropna()

In [30]:
scores2_df['DayofMonth'] = pd.to_datetime(scores2_df['schedule_date']).dt.day
scores2_df['Month'] = pd.to_datetime(scores2_df['schedule_date']).dt.month
scores2_df['year'] = pd.to_datetime(scores2_df['schedule_date']).dt.year

In [31]:
scores2_df

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,DayofMonth,Month,year
350,1/14/1968,1967,Superbowl,True,Green Bay Packers,33,14,Oakland Raiders,GB,-13.5,43,Orange Bowl,True,60.0,12.0,14,1,1968
538,1/12/1969,1968,Superbowl,True,Baltimore Colts,7,16,New York Jets,IND,-18.0,40,Orange Bowl,True,66.0,12.0,12,1,1969
727,1/11/1970,1969,Superbowl,True,Kansas City Chiefs,23,7,Minnesota Vikings,MIN,-12.0,39,Tulane Stadium,True,55.0,14.0,11,1,1970
916,1/17/1971,1970,Superbowl,True,Baltimore Colts,16,13,Dallas Cowboys,IND,-2.5,36,Orange Bowl,True,59.0,11.0,17,1,1971
1105,1/16/1972,1971,Superbowl,True,Dallas Cowboys,24,3,Miami Dolphins,DAL,-6.0,34,Tulane Stadium,True,34.0,18.0,16,1,1972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12655,12/29/2019,2019,17,False,Dallas Cowboys,47,16,Washington Redskins,DAL,-12.5,47.5,AT&T Stadium,False,72.0,0.0,29,12,2019
12657,12/29/2019,2019,17,False,Detroit Lions,20,23,Green Bay Packers,GB,-13.0,44,Ford Field,False,72.0,0.0,29,12,2019
12658,12/29/2019,2019,17,False,Houston Texans,14,35,Tennessee Titans,TEN,-10.0,43,NRG Stadium,False,72.0,0.0,29,12,2019
12667,1/4/2020,2019,Wildcard,True,Houston Texans,22,19,Buffalo Bills,HOU,-2.5,43,NRG Stadium,False,72.0,0.0,4,1,2020


## --------------------------------

In [32]:
stadiums_df.head()

,stadium_name,stadium_location,stadium_open,stadium_close,stadium_type,stadium_address,stadium_weather_station_code,stadium_weather_type,stadium_capacity,stadium_surface,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION
0,Alamo Dome,"San Antonio, TX",NaN,NaN,indoor,"100 Montana St, San Antonio, TX 78203",78203,dome,72000,FieldTurf,NaN,NaN,NaN,NaN,NaN
1,Alltel Stadium,"Jacksonville, FL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alumni Stadium,"Chestnut Hill, MA",NaN,NaN,outdoor,"Perimeter Rd, Chestnut Hill, MA 02467",2467,cold,NaN,Grass,NaN,NaN,NaN,NaN,NaN
3,Anaheim Stadium,"Anaheim, CA",1980.0,1994.0,outdoor,"2000 E Gene Autry Way, Anaheim, CA 92806",92806,warm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arrowhead Stadium,"Kansas City, MO",1972.0,NaN,outdoor,"1 Arrowhead Dr, Kansas City, MO 64129",64129,cold,"76,416",Grass,US1MOJC0028,"KANSAS CITY 5.1 SE, MO US",39.0692,-94.4871,264.9


In [33]:
stadiums_df.isna().mean()

stadium_name                    0.00
stadium_location                0.00
stadium_open                    0.24
stadium_close                   0.59
stadium_type                    0.07
stadium_address                 0.09
stadium_weather_station_code    0.10
stadium_weather_type            0.07
stadium_capacity                0.61
stadium_surface                 0.47
STATION                         0.48
NAME                            0.48
LATITUDE                        0.48
LONGITUDE                       0.48
ELEVATION                       0.48
dtype: float64

In [34]:
cols = ['stadium_name', 'stadium_location', 'stadium_type', 'stadium_weather_type']
stadiums_df = stadiums_df[cols]

In [35]:
stadiums_df.isna().mean()

stadium_name            0.00
stadium_location        0.00
stadium_type            0.07
stadium_weather_type    0.07
dtype: float64

In [36]:
stadiums_df['stadium_type'] = stadiums_df['stadium_type'].fillna('unknown')
stadiums_df['stadium_weather_type'] = stadiums_df['stadium_weather_type'].fillna('unknown')

In [37]:
stadiums_df.isna().mean()

stadium_name            0.0
stadium_location        0.0
stadium_type            0.0
stadium_weather_type    0.0
dtype: float64

## --------------------------------

In [38]:
teams_df.head(45)

,team_name,team_name_short,team_id,team_id_pfr,team_conference,team_division,team_conference_pre2002,team_division_pre2002
0,Arizona Cardinals,Cardinals,ARI,CRD,NFC,NFC West,NFC,NFC West
1,Phoenix Cardinals,Cardinals,ARI,CRD,NFC,NaN,NFC,NFC East
2,St. Louis Cardinals,Cardinals,ARI,ARI,NFC,NaN,NFC,NFC East
3,Atlanta Falcons,Falcons,ATL,ATL,NFC,NFC South,NFC,NFC West
4,Baltimore Ravens,Ravens,BAL,RAV,AFC,AFC North,AFC,AFC Central
5,Buffalo Bills,Bills,BUF,BUF,AFC,AFC East,AFC,AFC East
6,Carolina Panthers,Panthers,CAR,CAR,NFC,NFC South,NFC,NFC West
7,Chicago Bears,Bears,CHI,CHI,NFC,NFC North,NFC,NFC Central
8,Cincinnati Bengals,Bengals,CIN,CIN,AFC,AFC North,AFC,AFC Central
9,Cleveland Browns,Browns,CLE,CLE,AFC,AFC North,AFC,AFC Central


In [39]:
teams_df.shape

(41, 8)

In [40]:
teams_df.isna().mean()

team_name                  0.000000
team_name_short            0.000000
team_id                    0.000000
team_id_pfr                0.000000
team_conference            0.000000
team_division              0.195122
team_conference_pre2002    0.000000
team_division_pre2002      0.048780
dtype: float64

In [41]:
teams_df['team_division'] = teams_df['team_division'].fillna(teams_df['team_division_pre2002'])
teams_df['team_division'] = teams_df['team_division'].fillna('unknown')

In [42]:
teams_df = teams_df.drop(columns=['team_conference_pre2002', 'team_division_pre2002'])

In [44]:
teams_df

,team_name,team_name_short,team_id,team_id_pfr,team_conference,team_division
0,Arizona Cardinals,Cardinals,ARI,CRD,NFC,NFC West
1,Phoenix Cardinals,Cardinals,ARI,CRD,NFC,NFC East
2,St. Louis Cardinals,Cardinals,ARI,ARI,NFC,NFC East
3,Atlanta Falcons,Falcons,ATL,ATL,NFC,NFC South
4,Baltimore Ravens,Ravens,BAL,RAV,AFC,AFC North
5,Buffalo Bills,Bills,BUF,BUF,AFC,AFC East
6,Carolina Panthers,Panthers,CAR,CAR,NFC,NFC South
7,Chicago Bears,Bears,CHI,CHI,NFC,NFC North
8,Cincinnati Bengals,Bengals,CIN,CIN,AFC,AFC North
9,Cleveland Browns,Browns,CLE,CLE,AFC,AFC North


## Join dataframes together

In [43]:
full_table = hot_100.merge(spotify, left_on="SongID", right_on="SongID")

NameError: name 'hot_100' is not defined

## 3) Model your outcome of interest. You should try several different approaches and really work to tune a variety of models before using the model evaluation techniques to choose what you consider to be the best performer. Make sure to think about explanatory versus predictive power, and experiment with both.

## Execute the three tasks above in a Jupyter Notebook that you will submit to the grading team.

## Next, to prepare for your presentation, create a slide deck and a 15-minute presentation that guides viewers through your model. Be sure to cover a few specific topics:

* ## A specified research question that your model addresses
* ## How you chose your model specification and what alternatives you compared it to
* ## The practical uses of your model for an audience of interest
* ## Any weak points or shortcomings of your model

## This presentation is not a drill. You'll be presenting this slide deck live to a group as the culmination of all your work so far on supervised learning. As a secondary matter, your slides and the Jupyter Notebook should be worthy of inclusion as examples of your work product when applying to jobs.